In [51]:
%matplotlib inline

In [52]:
import numpy as np
from scipy import optimize
import matplotlib

The $Q$ matrix is written in the following format:

$X_{i,L_k} , X_{i,L_k-1} , X_{i,L_k-2} , ... , X_{i, 1} , G_{i,L_k} , G_{i,L_k-1} , G_{i,L_k-2} , ... , G_{i, 1}$

In [119]:
# Problem arguments
L = [3,3]
N = 2
m = 0
for k in L:
    m += k*2
print "Q matrix dimension: ", m

Q matrix dimension:  12


In [99]:
# Build the hessian matrix

Q = np.zeros((m, m))
off = 0
for o in xrange(N):
    off = o*L[o-1]*2
    for i in xrange(L[o]):
        # Insert elements in the diagonal
        if i == 0:
            Q[i+off][i+off] = 1.
            Q[i+off][i+off+1] = -2.
            Q[i+off+1][i+off] = -2.
            Q[i+off][i+off+L[o]] = -2.                
            Q[L[o]+i+off][i+off] = -2.

            Q[L[o]+i+off][L[o]+i+off] = 1. + 1.
        elif i == L[o]-1:
            Q[i+off][i+off] = L[o]

            Q[i+off][i+off+L[o]-1] = 2.
            Q[i+off+L[o]-1][i+off] = 2.

            Q[i+off][i+off+L[o]] = -2. * (L[o]-1)
            Q[i+off+L[o]][i+off] = -2. * (L[o]-1)

            Q[L[o]+i+off][L[o]+i+off] = L[o]-1 + 1
        else:
            Q[i+off][i+off]   =  2.
            Q[i+off][i+off+1] = -2.
            Q[i+off+1][i+off] = -2.

            Q[i+off][i+off+L[o]-1] = 2
            Q[i+off][i+off+L[o]] = -2.

            Q[i+off+L[o]-1][i+off] = 2.
            Q[i+off+L[o]][i+off] = -2.

            Q[L[o]+i+off][L[o]+i+off]=1+1

In [100]:
# Build the constants array
C = np.zeros(m)
off = 0
for o in L:
    for j in xrange(o*2):
        if j == 0:
            C[j+off] = -2.
        if j == o - 1:
            C[j+off] = 2.
        if j > o-1 and j < 2*o-1:
            C[j+off] = 2.
    off += o*2

In [101]:
C0 = 0
for k in L:
    C0 += k

In [102]:
print "Q = \n", Q
print "C = \n", C
print "C0 = \n", C0

H = 
[[ 1. -2.  0. -2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-2.  2. -2.  2. -2.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -2.  3.  0.  2. -4.  0.  0.  0.  0.  0.  0.]
 [-2.  2.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -2.  2.  0.  2.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -4.  0.  0.  3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -2.  0. -2.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -2.  2. -2.  2. -2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -2.  3.  0.  2. -4.]
 [ 0.  0.  0.  0.  0.  0. -2.  2.  0.  2.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -2.  2.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -4.  0.  0.  3.]]
C = 
[-2.  0.  2.  2.  2.  0. -2.  0.  2.  2.  2.  0.]
C0 = 
6


In [115]:
# Start the optimisation at a random point
x0 = [0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0]

def loss(x, sign=1.):
    return sign * (0.5 * np.dot(x.T, np.dot(Q, x)) + np.dot(C, x) + C0 )

def jac(x, sign=1.):
    return sign * (np.dot(x.T,Q) + C)

opt = {'disp': True, 'maxiter': 10000}

In [117]:
res_uncons = optimize.minimize(loss, x0, method='Newton-CG',
                              jac=jac, options=opt)

         Current function value: -7071439762914200346119883824373131731925676519678867182273763322423218866574875890550238494374523400924463579587862587750807851635108065821063143576360528691699945058722343890755039920776916724170603626165018592565809970178474485772650976770642852641489156687278407260830972271459648271489206627755622400.000000
         Iterations: 117
         Function evaluations: 920
         Gradient evaluations: 1340
         Hessian evaluations: 0


In [120]:
print "\nUnconstrained:"
print res_uncons


Unconstrained:
     fun: -7.0714397629142003e+303
     jac: array([ -3.12027146e+151,  -4.47628375e+151,  -9.12996054e+151,
         1.74251218e+151,   4.39338680e+150,  -5.18447251e+151,
        -3.12027146e+151,  -4.47628375e+151,  -9.12996054e+151,
         1.74251218e+151,   4.39338680e+150,  -5.18447251e+151])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 920
    nhev: 0
     nit: 117
    njev: 1340
  status: 2
 success: False
       x: array([  1.37775928e+151,   2.88972862e+151,   3.80352213e+151,
        -6.40713255e+150,  -6.94124167e+150,   3.34320534e+151,
         1.37775928e+151,   2.88972862e+151,   3.80352213e+151,
        -6.40713255e+150,  -6.94124167e+150,   3.34320534e+151])
